In [1]:
import os
import glob
import pandas as pd

In [2]:
pwd

'/Users/michaelpiacentino/Drive/git/data/smpd3/smpd3_dataset/20180616_gRNA1_Pax7_Laminin_TCFLef/sec/pax7counts'

In [4]:
# Add experiment date here to apply to dataframe
expt_date = '20180616'
treatment = 'SMPD3gRNA1'
result_type = 'Pax7Counts'

path = os.path.abspath('')+'/pax7countCSV/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Date'] = expt_date
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Parse Image name
    df['Target'], df['Treatment'], df['Embryo'], df['Section'], df['MIP'] = zip(*df['Image'].map(lambda x: x.split('_')))

    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,Slice,Count,Total Area,Average Size,%Area,Mean,IntDen,Date,Image,Target,Treatment,Embryo,Section,MIP
0,CntlSide,19,1502.998,79.105,19.112,255,20171.817,20180616,SMPD3_CRISPR_Emb9_sec3_MIP,SMPD3,CRISPR,Emb9,sec3,MIP
1,ExptSide,19,1209.747,63.671,19.602,255,16236.072,20180616,SMPD3_CRISPR_Emb9_sec3_MIP,SMPD3,CRISPR,Emb9,sec3,MIP
0,CntlSide,58,6808.837,117.394,26.243,255,29935.404,20180616,SMPD3_CRISPR_Emb3_sec1_MIP,SMPD3,CRISPR,Emb3,sec1,MIP
1,ExptSide,48,3714.572,77.387,21.639,255,19733.662,20180616,SMPD3_CRISPR_Emb3_sec1_MIP,SMPD3,CRISPR,Emb3,sec1,MIP
0,CntlSide,36,2686.362,74.621,18.787,255,19028.396,20180616,SMPD3_CRISPR_Emb4_sec1_MIP,SMPD3,CRISPR,Emb4,sec1,MIP


In [5]:
# Mean counts across sections
mean_sections = pd.DataFrame((full_df.groupby(['Embryo', 'Slice'])['Count']).mean())
mean_sections
# Extract means for Cntl and Expt counts
pax7_cntl = mean_sections.xs('CntlSide', level='Slice')['Count'] 
pax7_expt = mean_sections.xs('ExptSide', level='Slice')['Count']

# Generate ratios as Expt/Cntl
pax7_ratios = pd.DataFrame(pax7_expt / pax7_cntl)
pax7_ratios.columns = ['Expt/Cntl Pax7']

In [6]:
# Combine processed values into single dataframe and output as csv file
pax7_cntl = pd.DataFrame(pax7_cntl)
pax7_cntl.columns = ['Cntl Pax7']
pax7_expt = pd.DataFrame(pax7_expt)
pax7_expt.columns = ['Expt Pax7']

results = pd.concat([pax7_cntl, pax7_expt, pax7_ratios], axis=1, sort=True)
results['ID'] = expt_date + '_' + results.index
results.to_csv(expt_date + '_' + treatment + '_' + result_type + '_results.csv')